# 1. Importing Libraries including custom made

In [1]:
import os 
import subprocess
# Change the current working directory
os.chdir("C:\\Users\\Jayedi Aman\\Downloads\\BEM\\pcm_paper")

from pcm_class import IDF_EDITING
from pcm_block import MaterialProperty
from copy_to_path import copy_to_dir

# 2. Copy file from one directorty and added extension to name and copied to other directory

In [2]:
import os
# Paths
dir1 = r"C:\Users\Jayedi Aman\Downloads\BEM\ASHRAE901_OfficeMedium_STD2013"
dir2 = r"C:\Users\Jayedi Aman\Downloads\BEM\ASHRAE_SO_2013\mediumPCM28"
copy_to_dir(dir1, dir2, file_type='idf', extension='_pcm28.')

##  Testing with one file
# dir1 = r"C:\Users\Jayedi Aman\Downloads\BEM\pcm_paper\pcm_idf"
# dir2 = r"C:\Users\Jayedi Aman\Downloads\BEM\pcm_paper\pcm_idf"
# copy_to_dir(dir1, dir2)

# 3. User input for material proprties 

In [9]:
## appending pcm properties to idf
# file01 = ""
_envelope_name = 'nonres_ext_wall_grd'
# _envelope_name="WALL-1"
_split_word = 'Layer ' # Where PCM layer will be added
_pcm_layer = "!- Layer 3"


# Example to use
pcm_mat = MaterialProperty('pcmboard')
# PCM23
# pcm_mat.material(0.025, 0.27, 1000, 2120)
# pcm_mat.phasechange(-20.0, 0, 23, 55000, 24, 75000, 100, 165000)
#PCM28
pcm_mat.material(0.025, 0.25, 814, 2150)
pcm_mat.phasechange(-20.0, 0, 28, 103200, 28.1, 226300, 100, 481000)
_pcm_block = pcm_mat.pcmblock()

# 4. Finding idf file names in the directory

In [10]:
## List of idf in a folder
import os
def idf_file_list(file_path):
    # Import necessary libraries
    import os, glob, shutil
    # Change directory to old idf files to work with
    os.chdir(file_path)

    # Getting names of all idf files only
    files = glob.glob("*.idf")
    return files

file_path = r"C:\Users\Jayedi Aman\Downloads\BEM\ASHRAE_SO_2013\mediumPCM28"
idf_files = idf_file_list(file_path)
print(idf_files)

['ASHRAE901_OfficeMedium_STD2013_Albuquerque_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Atlanta_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Buffalo_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Denver_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_ElPaso_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_GreatFalls_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Miami_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_NewYork_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_PortAngeles_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Rochester_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_SanDiego_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Seattle_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Tampa_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Tucson_pcm28.idf']


# 5. Editing idf files with PCM properties

In [11]:
for ii in range(len(idf_files)):
    file01 = os.path.abspath(idf_files[ii])
    file_path = file01
        
    idf_final_dict = {}

    file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer,  _pcm_block)
    
    # Editing entire idf file
    file_object.create_idf()

    # # Editing only PCM properties. If need to add new PCM layer to other walls
    # file_object.create_idf__no_appending()


# ## Testing with one file only so loop is commented out  
# file01 = os.path.abspath(idf_files[1])
# file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer,  _pcm_block)
# file_object.create_idf()

# 6. Creating new directories for each idf file and run simulation 

In [12]:
import os
import subprocess

def find_matching_epw(epw_directory, idf_filename):
    base_name = idf_filename.replace("_pcm28", "").split('_')[-1].split('.')[0]
    # base_name = idf_filename.replace("_pcm23", "").split('_')[-1].split('.')[0]
    
    for epw_file in os.listdir(epw_directory):
        if epw_file.endswith('.epw') and base_name.lower() in epw_file.lower():
            return os.path.join(epw_directory, epw_file)
    return None

def run_energyplus(idf_directory, epw_directory, eplus_dir):
    for file_name in os.listdir(idf_directory):
        if file_name.endswith('.idf'):
            idf_filepath = os.path.join(idf_directory, file_name)
            base_name = os.path.splitext(file_name)[0]
            output_directory = os.path.join(idf_directory, f"results_{base_name}")

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            matched_epw = find_matching_epw(epw_directory, file_name)
            if matched_epw:
                cl_st = [
                    f"{eplus_dir}\\EnergyPlus",
                    "--readvars",
                    f"--output-directory={output_directory}",
                    f"--weather={matched_epw}",
                    idf_filepath,
                ]

                result = subprocess.run(cl_st, capture_output=True, text=True)

                if result.returncode == 0:
                    print(f"Simulation for {file_name} completed successfully.")
                else:
                    print(f"Simulation for {file_name} failed.")
                    print("Error:", result.stderr)
            else:
                print(f"No matching .epw file found for {file_name}.")

idf_directory = r'C:\Users\Jayedi Aman\Downloads\BEM\ASHRAE_SO_2013\mediumPCM28'  # Path to .idf files
epw_directory = r'C:\Users\Jayedi Aman\Downloads\BEM\ASHRAE901_epw'  # Path to .epw files
eplus_dir = r'C:\EnergyPlusV22-1-0'  # Path to EnergyPlus installation
 
run_energyplus(idf_directory, epw_directory, eplus_dir)

Simulation for ASHRAE901_OfficeMedium_STD2013_Albuquerque_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Atlanta_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Buffalo_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Denver_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_ElPaso_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_GreatFalls_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Miami_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_NewYork_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_PortAngeles_pcm28.idf completed successfully.
Simulation f

# F8 Metal deleted from Medium Office